In [1]:
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs



e:\Code\Multi_Task_Federated_Learning\env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ratings = tfds.load('movielens/100k-ratings', split="train")
movies = tfds.load('movielens/100k-movies', split="train")

# Select the basic features.
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
    "user_rating": x["user_rating"],
})
movies = movies.map(lambda x: x["movie_title"])


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [3]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

movie_titles = movies.batch(1_000)
user_ids = ratings.batch(1_000_000).map(lambda x: x["user_id"])

unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

In [4]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

movie_titles = movies.batch(1_000)
user_ids = ratings.batch(1_000_000).map(lambda x: x["user_id"])

unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

In [5]:
user_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_user_ids, mask_token=None),
  # We add 1 to account for the unknown token.
  tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
])

movie_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_movie_titles, mask_token=None),
  tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
])

NameError: name 'embedding_dimension' is not defined

In [ ]:
tfrs.tasks.Ranking(
    loss=tf.keras.losses.MeanSquaredError(),
    metrics=[tf.keras.metrics.RootMeanSquaredError()],
)

In [ ]:
tfrs.tasks.Retrieval(
    metrics=tfrs.metrics.FactorizedTopK(
        candidates=movies.batch(128)
    )
)

In [6]:
user_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_user_ids, mask_token=None),
  # We add 1 to account for the unknown token.
  tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
])

movie_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_movie_titles, mask_token=None),
  tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
])

In [8]:
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

In [12]:
model = MovielensModel(rating_weight=1.0, retrieval_weight=1.0)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

In [13]:
model.fit(cached_train, epochs=3)
metrics = model.evaluate(cached_test, return_dict=True)

print(f"Retrieval top-100 accuracy: {metrics['factorized_top_k/top_100_categorical_accuracy']:.3f}.")
print(f"Ranking RMSE: {metrics['root_mean_squared_error']:.3f}.")

Epoch 1/3
10/10 [==============================] - 13s 1s/step - root_mean_squared_error: 2.0694 - factorized_top_k/top_1_categorical_accuracy: 0.0013 - factorized_top_k/top_5_categorical_accuracy: 0.0099 - factorized_top_k/top_10_categorical_accuracy: 0.0223 - factorized_top_k/top_50_categorical_accuracy: 0.1011 - factorized_top_k/top_100_categorical_accuracy: 0.1774 - loss: 69870.1974 - regularization_loss: 0.0000e+00 - total_loss: 69870.1974
Epoch 2/3
10/10 [==============================] - 11s 1s/step - root_mean_squared_error: 1.3028 - factorized_top_k/top_1_categorical_accuracy: 0.0029 - factorized_top_k/top_5_categorical_accuracy: 0.0201 - factorized_top_k/top_10_categorical_accuracy: 0.0400 - factorized_top_k/top_50_categorical_accuracy: 0.1710 - factorized_top_k/top_100_categorical_accuracy: 0.2949 - loss: 67461.3047 - regularization_loss: 0.0000e+00 - total_loss: 67461.3047
Epoch 3/3
5/5 [==============================] - 3s 464ms/step - root_mean_squared_error: 1.1167 - fac

In [28]:
trained_movie_embeddings, trained_user_embeddings, predicted_rating = model({
      "user_id": np.array(["42"]),
      "movie_title": np.array(["Dances with Wolves (1990)"])
  })
print("Predicted rating:")
print(predicted_rating)
print(trained_user_embeddings)
print(trained_movie_embeddings)

Predicted rating:
tf.Tensor([[3.3569176]], shape=(1, 1), dtype=float32)
tf.Tensor(
[[-0.21009754 -0.34264356  0.10703869 -0.28632584  0.28924102  0.33073252
   0.10074507  0.38770652  0.1938414   0.09020282 -0.2200806   0.11886927
  -0.13905436 -0.16398759  0.2505125   0.27448255 -0.36408946 -0.219865
   0.00802449  0.2159886  -0.39927325 -0.23138468  0.34872943 -0.24597391
   0.20023398 -0.27112496  0.39964023 -0.49603942  0.10990362  0.349834
  -0.13639118 -0.19449641]], shape=(1, 32), dtype=float32)
tf.Tensor(
[[-0.08064345  0.34927124  0.4053641  -0.752307    0.16353123  0.09554704
  -0.10276742  0.04004689  0.02206565  0.3805118   0.46147352 -0.3647723
  -0.18874076 -0.27783915  0.4673536   0.31899333 -0.43886188 -0.06296811
  -0.28117844  0.14490582 -0.5155211  -0.12255418 -0.34890306 -0.21599993
   0.08883139 -0.19850881  0.35075533 -0.02203721 -0.15130411  0.25241414
  -0.5650145  -0.43493795]], shape=(1, 32), dtype=float32)


In [33]:
# Create a model that takes in raw query features, and
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
# recommends movies out of the entire movies dataset.
index.index_from_dataset(
  tf.data.Dataset.zip((movies.batch(100), movies.batch(100).map(model.movie_model)))
)

# Get recommendations.
_, titles = index(tf.constant(["3"]))
print(f"Recommendations for user 42: {titles[0, :3]}")

Recommendations for user 42: [b'Deep Rising (1998)' b'Fallen (1998)' b'Sphere (1998)']


In [ ]:
tf.saved_model.save(index, )